## Tarefa - Revisão de generalização
### Ajuste de características - Titanic Competition


In [29]:
# Desabilita warnings
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
# train = pd.read_csv('train.csv')
# test = pd.read_csv('test.csv')
train = pd.read_csv('datasets/titanic/train.csv')
test = pd.read_csv('datasets/titanic/test.csv')
X = train[list(test.columns)]
y = train[train.columns[~train.columns.isin(test.columns)]]


In [30]:
from sklearn.base import BaseEstimator, TransformerMixin

def extraiPronome(nome):
    return nome.split(',')[1].split('.')[0].strip()

# Remome colunas indesejadas no final do tratamento dos dados
class AtributosDesejados(BaseEstimator, TransformerMixin):
    def __init__(self, excluirName=True):
        self.excluirName = excluirName
    def fit(self, X, y=None):
        self.colunasIndesejadas = ['PassengerId', 'Ticket', 'Cabin']
        if self.excluirName:
            self.colunasIndesejadas.append('Name')
        return self
    def transform(self, X, y=None):
        if 'Name' not in self.colunasIndesejadas:
            X['Name'] = X['Name'].apply(extraiPronome)
            
            # Trata pronome de tratamento
            X['Name'] = X['Name'].replace('Mlle', 'Miss')
            X['Name'] = X['Name'].replace(['Ms','Mme'] , 'Mrs')
            X['Name'] = X['Name'].replace(['Lady', 'the Countess', 'Dona'] , 'Mrs')
            X['Name'] = X['Name'].replace(['Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer'], 'Mr')
            #X['Name'] = X['Name'].replace(['Lady', 'the Countess', 'Dona', 
            #                                'Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer'], 'Other')

        
        # Deck (letra inicial da cabine)
        X['Deck'] = X["Cabin"].str.slice(0,1)        

        # Classe para 'Sex'
        X['Sex_Class'] = 0
        X.loc[ X['Sex'] == 'female', 'Sex_Class'] = 0
        X.loc[ X['Sex'] == 'male', 'Sex_Class'] = 1
        X['Sex_Class'] = X['Sex_Class'].astype(int)


        # Classes para as idades 
        X['Age_Class'] = ''
        X['Age'].fillna(X['Age'].median(), inplace=True)
        
        X.loc[ X['Age'] < 16, 'Age_Class'] = 'children'
        X.loc[(X['Age'] >= 16) & (X['Age'] < 35), 'Age_Class'] = 'young'
        X.loc[(X['Age'] > 35) & (X['Age'] < 65), 'Age_Class'] = 'adult'
        X.loc[ X['Age'] >= 65, 'Age_Class'] = 'old'


        # Classes para tarifa
        X['Fare_Class'] = ''
        X['Fare'].fillna(X['Fare'].median(), inplace=True)

        X.loc[ X['Fare'] < 25, 'Fare_Class'] = 'low_fare'
        X.loc[(X['Fare'] >= 20) & (X['Fare'] < 50), 'Fare_Class'] = 'median_fare'
        X.loc[(X['Fare'] >= 50) & (X['Fare'] < 100), 'Fare_Class'] = 'median_high_fare'
        X.loc[ X['Fare'] >= 100, 'Fare_Class'] = 'high_fare'


        # Cria característica FamilySize
        X['FamilySize'] = 0
        X['FamilySize'] = X['SibSp'] + X['Parch'] + 1
        X['FamilySize'] = X['FamilySize'].astype(int)

        # Classe para o tamanho da família
        X['FamilySize_class'] = ''
        X.loc[ X['FamilySize'] == 1, 'FamilySize_class'] = 'alone'
        X.loc[(X['FamilySize'] > 1) & (X['FamilySize'] <= 4 ), 'FamilySize_class'] = 'small'
        X.loc[(X['FamilySize'] > 4) & (X['FamilySize'] <= 7), 'FamilySize_class'] = 'medium'
        X.loc[ X['FamilySize'] > 7, 'FamilySize_class'] = 'large'     
    
        # # Classifica pessoas viajando sozinhas e cria nova característica 'isAlone'
        # X['isAlone'] = 0
        # X.loc[(X['SibSp'] == 0) & (X['Parch'] == 0), 'isAlone'] = 1
        # X.loc[(X['SibSp'] > 0) | (X['Parch'] > 0), 'isAalone'] = 0
        # X['isAlone'] = X['isAlone'].astype(int)

        # Remove colunas indesejadas
        X = X.drop(self.colunasIndesejadas,axis=1)
        
        return X


In [31]:
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosNumericos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasNumericas = X.select_dtypes(include='number').columns
        return self
    def transform(self, X, y=None):
        return X[self.colunasNumericas].to_numpy()


In [32]:
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosCategoricos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasCategoricas = X.select_dtypes(include='object').columns
        return self
    def transform(self, X, y=None):
        return X[self.colunasCategoricas].to_numpy()


In [33]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion

trataAtributos = Pipeline([
    ('unecaracteristicas', FeatureUnion([
        ('pipenum', Pipeline([
            ('atributos_numericos', AtributosNumericos()),
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ])),
        ('pipecat', Pipeline([
            ('atributos_categoricos', AtributosCategoricos()),
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]))
    ])),
])


In [34]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_validate, RepeatedKFold
import numpy as np

pipetotal = Pipeline([
    ('atributosDesejados', AtributosDesejados()),
    ('trataAtributos', trataAtributos),
    ('classificador', RandomForestClassifier())
])

parametros = {
    'atributosDesejados__excluirName': [False],
    'classificador__max_depth': [5],
    #'classificador__max_depth': [3, 5, 7, 9, 11, 13, 15 ,25],
    #'classificador__class_weight': ['balanced'],
    #'classificador__criterion': ['gini', 'entropy'],
    #'classificador__max_features': [None, 2, 4, 6, 'sqrt'],
    #'classificador__n_estimators': [100, 125, 150],
    #'classificador__min_samples_split': [3, 4, 5, 6, 8, 10],
    #'classificador__min_samples_leaf': [2, 3, 4]
}

modelo = GridSearchCV(pipetotal, param_grid=parametros, scoring='average_precision', n_jobs=-1)

scores = cross_validate(modelo, X, y, cv=RepeatedKFold())
scores['test_score'], np.mean(scores['test_score']), np.std(scores['test_score'])

(array([0.81988684, 0.84094741, 0.84707917, 0.86650867, 0.8632715 ,
        0.88430714, 0.87233819, 0.73657397, 0.8890153 , 0.84693894,
        0.90031701, 0.8527718 , 0.82709141, 0.81382698, 0.85700222,
        0.88670753, 0.8657576 , 0.83084526, 0.80224084, 0.90369267,
        0.83689894, 0.8707759 , 0.85168063, 0.87301649, 0.82514635,
        0.86485886, 0.87063115, 0.83144061, 0.83072138, 0.87250455,
        0.85393777, 0.83864025, 0.8771233 , 0.81866142, 0.87503339,
        0.86185127, 0.91162287, 0.83282307, 0.84490155, 0.80044094,
        0.7772265 , 0.83364468, 0.88498557, 0.89251788, 0.86021222,
        0.88403488, 0.84161792, 0.90412101, 0.84150735, 0.79921962]),
 0.8513783755240538,
 0.03385463279024151)

In [35]:
modelo.fit(X,y)
y_pred = modelo.predict(test)
result = test[['PassengerId']]
result['Survived'] = y_pred
result.to_csv('submission.csv',index=False)